In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import os
import json
import re
%matplotlib inline
DATA_DIR = './datas/datas'

## 数据描述

### 数据集简介
[数据库的历史](https://projects.iq.harvard.edu/chinesecbdb/%E6%AD%B7%E5%8F%B2)

> 郝若貝教授的目標在於利用中國歷史紀錄中保存最豐富之資料(包括公領域之資料如官方歷史記載，及私領域之資料如墓誌銘)，重現單一個人於多脈絡中之面貌：例如，個人作為中心地或行政區劃的本地人或居住者、具有一定的官品和官職，以及作為家族網絡中的一分子。研究者得以透過此一資料庫，探討某特定地方在科舉和出仕方面的表現如何，或將某特定時空下的個人之創作著述聯接起來，或作為其他研究目的之用 

也就是说，其希望建设的是历史人物数据库，个人为中心，具有一定的官品和官职，并且作为家族网络中的一份子。
由此说明，该数据库是**个人中心**，具有一定的**社会地位**，**家族网络**

其提供了sqlite,access的数据库和api，由于相比数据库的字段繁多的，我选择了api，于是写了一个爬虫，便利personid得到相关的数据(422271)。

数据以人物组织，在给出的[王安石的例子](https://cbdb.fas.harvard.edu/cbdbapi/person.php?id=1762)中，可以看到其包含王安石的生平信息，出生，死亡，入仕、社會區分、亲戚关系等。其中值得一提的是，社會關係关系中包含许多的关系，包括推荐、欣賞/器重等正面的信息外，还包括其政策被Y反對/不支持等负面的信息，这部分信息结合书中第5章关于正负关系讨论，是得我对这个sign graph非常的好奇。


In [2]:
def basic_statistic():
    '''遍历data文件夹，统计那些PersonInfo不为空的数据'''
    counts = 0
    for fname in os.listdir(DATA_DIR):
        fpath = os.path.join(DATA_DIR, fname)
        with open(fpath) as f:
            try:
                json_data = json.load(f)
                person_data = json_data['Package']['PersonAuthority']['PersonInfo']['Person']
                counts += 1
            except Exception as e:
                pass
#                 print(fpath, e)
    print('total: ', counts)
# basic_statistic()

In [48]:
def basic_statistic_network():
    '''基本统计network信息'''
    edge_lists = []
    counts = 0
    t = 0
    for fname in os.listdir(DATA_DIR):
        fpath = os.path.join(DATA_DIR, fname)
        with open(fpath) as f:
            try:
                json_data = json.load(f)
                person_data = json_data['Package']['PersonAuthority']['PersonInfo']['Person']
                t += 1
                if 'PersonSocialAssociation' in person_data:
                    person_association = person_data["PersonSocialAssociation"]
                    if 'Association' in person_association:
                        person_assoc = person_association['Association']
                        if isinstance(person_assoc, list):
                            for person in person_association['Association']:
                                if 'AssocPersonId' in person:
                                    #print(fname[:fname.rfind('.')], person['AssocPersonId'])
                                    edge_lists.append((fname[:fname.rfind('.')], person['AssocPersonId']))
                                #if count > 10:break
                                    counts += 1
                        elif isinstance(person_assoc, dict):
                            #print('dict:', fname[:fname.rfind('.')], person_assoc['AssocPersonId'])
                            edge_lists.append((fname[:fname.rfind('.')], person_assoc['AssocPersonId']))
                            counts +=1 
            except Exception as e:
                pass
    print(t, counts, len(edge_lists))
    G = nx.Graph()
    G.add_edges_from(edge_lists)
    #nx.draw(G, with_labels=False, font_weight='bold')
    print(nx.info(G))

# basic_statistic_network()

'''
422271 131879 131879
Name: 
Type: Graph
Number of nodes: 27889
Number of edges: 48004
Average degree:   3.4425

'''


422271 131879 131879
Name: 
Type: Graph
Number of nodes: 27889
Number of edges: 48004
Average degree:   3.4425


## 朝代划分

由于整个网络的节点过多，且横跨的时间较长，因此选择根据朝代对数据进行划分。

其中需要一提的是，由于部分数据可能不详，因此通过以下三个原则对数据进行划分：

    1.标有的朝代
    2.出生年份在朝代年份内
    3.死亡年份在朝代年份内

In [47]:
def handle_dynasty():
    '''
    数据整理为朝代
    '''
    for dynasty in DYNASTY:
        if not os.path.exists('datas/{}'.format(dynasty)):
            os.mkdir('datas/{}'.format(dynasty))
    json_error_f = open('json_error.log', 'w')
    for fname in os.listdir(DATA_DIR):
        fpath = os.path.join(DATA_DIR, fname)
        with open(fpath) as f:
            try:
                json_data = json.load(f)
                person_data = json_data['Package']['PersonAuthority']['PersonInfo']['Person']
                for dynasty in DYNASTY:
                    if is_dynasty(dynasty, person_data, fname):
                        result_path = os.path.join('datas', dynasty, fname)
                        with open(result_path, 'w') as f2:
                            f2.write(json.dumps(person_data))
                        print(fname, dynasty)
            except Exception as e:
                print(fpath, e, file = json_error_f)

### 不同朝代的统计结果
提取PersonSocialAssociation的Association字段。形成网络，统计不同朝代的网络的节点数和连边数。

In [49]:
def read_dynasty(dynasty = '宋'):
    edge_lists = []
    count = 0
    t = 0
    dynasty_dir = os.path.join('datas', dynasty)
    for fname in os.listdir(dynasty_dir):
        t += 1
        fpath = os.path.join(dynasty_dir, fname)
        f = open(fpath)
        json_data = json.load(f)
        if 'PersonSocialAssociation' in json_data:
            person_association = json_data["PersonSocialAssociation"]
            if 'Association' in person_association:
                person_assoc = person_association['Association']
                if isinstance(person_assoc, list):
                    for person in person_association['Association']:
                        if 'AssocPersonId' in person:
                            #print(fname[:fname.rfind('.')], person['AssocPersonId'])
                            edge_lists.append((fname[:fname.rfind('.')], person['AssocPersonId']))
                        count +=1
                        #if count > 10:break
                elif isinstance(person_assoc, dict):
                    #print('dict:', fname[:fname.rfind('.')], person_assoc['AssocPersonId'])
                    edge_lists.append((fname[:fname.rfind('.')], person_assoc['AssocPersonId']))
    print(count, t)
    G = nx.Graph()
    G.add_edges_from(edge_lists)
    #nx.draw(G, with_labels=False, font_weight='bold')
    print(nx.info(G)) 
    return G

# print('唐朝')    
# read_dynasty('唐')

# print('宋朝')    
# read_dynasty('宋')

# print('元朝')    
# read_dynasty('元')

# print('明朝')    
# read_dynasty('明')

# print('清朝')    
# read_dynasty('清')


# 唐朝
# 288 50402
# Name: 
# Type: Graph
# Number of nodes: 365
# Number of edges: 286
# Average degree:   1.5671
# 宋朝
# 78652 53531
# Name: 
# Type: Graph
# Number of nodes: 17114
# Number of edges: 30330
# Average degree:   3.5445
# 元朝
# 23083 23228
# Name: 
# Type: Graph
# Number of nodes: 6424
# Number of edges: 11864
# Average degree:   3.6936
# 明朝
# 27534 189622
# Name: 
# Type: Graph
# Number of nodes: 8350
# Number of edges: 14609
# Average degree:   3.4992
# 清朝
# 3920 89435
# Name: 
# Type: Graph
# Number of nodes: 3128
# Number of edges: 3059
# Average degree:   1.9559
g = read_dynasty('唐')
nx.write_gexf(g, "tang.gexf")


唐朝
288 50402
Name: 
Type: Graph
Number of nodes: 365
Number of edges: 286
Average degree:   1.5671
宋朝
78652 53531
Name: 
Type: Graph
Number of nodes: 17114
Number of edges: 30330
Average degree:   3.5445
元朝
23083 23228
Name: 
Type: Graph
Number of nodes: 6424
Number of edges: 11864
Average degree:   3.6936
明朝
27534 189622
Name: 
Type: Graph
Number of nodes: 8350
Number of edges: 14609
Average degree:   3.4992
清朝
3920 89435
Name: 
Type: Graph
Number of nodes: 3128
Number of edges: 3059
Average degree:   1.9559


In [ ]:
def vis_network(nodes_num = 100, graph):
    '''
    实现网络的可视化，通过随机采样的方式对数据进行基本的可视化。
    当然更为合理的方式是将网络导入到gephi中。
    '''
    nodes = graph.nodes()
    samples = np.random.randint(0, len(nodes), nodes_num)
    samples_nodes = nodes[samples]
    subgraph = graph.subgraph(samples_nodes)

#### 将家族关系考虑进网络中的统计结果

有之前的统计结果可以看出，网络主要在宋元明。而弱不考虑家族关系的话，这个网络是社交网络。
但是其实应该将家族的网络加入到社会网络中，比较合理。



In [8]:
from collections import defaultdict
def statistic_relation():
    relations = set()
    relations_codes = set()
    relations_c = defaultdict(int)
    error = 0
    for fname in os.listdir(DATA_DIR):
        fpath = os.path.join(DATA_DIR, fname)
        with open(fpath) as f:
            try:
                json_data = json.load(f)
                if 'Person' not in json_data['Package']['PersonAuthority']['PersonInfo']:
                    continue
                person_data = json_data['Package']['PersonAuthority']['PersonInfo']['Person']
                if 'PersonSocialAssociation' in person_data:
                    person_association = person_data["PersonSocialAssociation"]
                    if 'Association' in person_association:
                        person_assoc = person_association['Association']
                        if isinstance(person_assoc, list):
                            for person in person_association['Association']:
                                if 'AssocName' in person and 'AssocCode' in person:
                                    relations.add(person['AssocName'])
                                    relations_codes.add(person['AssocCode'])
                                    tmp = (person['AssocName'], person['AssocCode'])
                                    relations_c[tmp] += 1
                        elif isinstance(person_assoc, dict):
                            relations.add(person['AssocName'])
                            relations_codes.add(person['AssocCode'])
                            tmp = (person['AssocName'], person['AssocCode'])
                            relations_c[tmp] += 1
            except json.decoder.JSONDecodeError as e:
                error += 1
    print(len(relations), len(relations_codes), len(relations_c))
    print(relations)
    print(relations_codes)
    for r in relations_c:
        print(r[0], ',', r[1], ',', relations_c[r])
statistic_relation()

string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
440 447 447
{'', '為Y之家僕', '其生祠由Y作記', '其（或追隨者）殺害Y', '為Y作齋、堂銘', '下屬為Y', '拒為Y掾屬', '其學得到Y之讚揚', '攻擊道學', '上司為Y', '教授Y', '以詩諷忤Y', '奪Y之妻', '被Y在遺表中推薦', '教授Y之學', '忌/惡', '鞫治', '游説Y被拒絕', '神道碑跋由Y所作', '為Y之生祠作記', '家僕為Y', '被Y推薦參加制科考試', '墓誌銘係由Y向他人（第三方）求得', '其政策得到Y的支持[併入26]', '逃離Y的統治區', '遺奏推薦', '不合', '稱道Y之文風', '齋、堂銘由Y所作', '為Y之潛邸舊人', '為Y作壙記', '軍隊得到Y之贍給', '求Y為他人（第三方）作神道碑', '諂事Y', '被Y陷害', '辟', '被Y逮捕', '為Y作諡議', '遭Y黨羽的攻訐', '被Y欣賞/器重', '遭Y反秦檜勢力排擠', '得到Y之幫助', '支持', '抵禦或討平叛軍Y', '與他的交往導致Y受牽連', '獻策於Y', '下令處決', '著述由Y刊刻', '為Y取名、字', '排Y之學', '收到Y的啓', '受Y之請作序、記', '從Y學', '與Y遊', '彈劾', '指教Y', '節行為Y所稱道', '家傳為Y所作', '收到Y的贈詩、文', '攻訐[併入15]', '以文章、學問受知於', '逮捕', '對他的赦免遭到Y的反對', '受經於Y', '由Y代作文', '反對/不支持Y的政策', '為Y之書、畫作跋', '贈Y物', '著作被Y評論', '同學、同門', '合撰(編)著作', '為Y之神道碑作額篆', '為Y作廟碑記', '職業被Y繼承', '求他人（第三方）為Y作神道碑', '為Y作祭文', '門客為Y', '為Y乞某事', '採Y之著述', '稱道Y之節行', '題Y的書帖', '為Y作碑陰', '其冤因

被Y逮捕 , 158 , 12
評論Y之著作 , 529 , 80
(暫時保留，待刪除) , 236 , 1
為Y作哀辭 [併入167] , 38 , 8
碑陰為Y所作 , 321 , 53
為使團副使Y的正使 , 279 , 13
拒為Y掾屬 , 260 , 27
同學、同門 , 117 , 357
被Y籌劃謀殺 , 60 , 19
與他的交往導致Y受牽連 , 244 , 42
其冤因Y得明 , 486 , 27
(暫時保留，待刪除) , 235 , 1
硯銘由Y所作 , 226 , 47
為Y作齋、堂銘 , 298 , 123
為Y作廟碑記 , 160 , 188
書序由Y所作 , 33 , 4774
為Y作硯銘 , 225 , 33
神道碑跋由Y所作 , 417 , 2
受經於Y , 217 , 33
為Y墓表作序 , 420 , 20
從Y學 , 231 , 419
名、字由Y所取 , 536 , 23
詩作為Y所稱道 , 70 , 94
為Y之行狀作跋 , 523 , 7
題Y之畫作 , 441 , 51
遭Y攻訐 [併入16] , 184 , 2
以文章、學問受知於 , 292 , 62
推薦 , 13 , 1171
行狀由Y所作 , 425 , 1313
哀辭由Y所作[併入166] , 39 , 10
其著作為Y所研讀 , 362 , 14
書之題詞由Y所作 , 581 , 284
為Y作畫贊(畫像記) , 428 , 571
逮捕 , 159 , 19
為Y之義女 , 549 , 3
為Y所著書作序 , 32 , 4323
黨羽為Y , 7 , 45
遭Y黨羽的攻訐 , 138 , 4
從Y至貶所 , 315 , 2
為Y之家譜作序 , 276 , 72
以宦官事Y , 354 , 5
友 , 9 , 1460
壙記由Y所作 , 462 , 87
喜爱 , 359 , 27
合撰(編)著作 , 251 , 125
被Y得罪 , 516 , 123
拒為Y之黨 , 303 , 4
收藏品由Y作跋 , 466 , 29
與Y攀親戚敘族屬 , 570 , 3
為Y之門人 , 19 , 490
其生祠由Y作記 , 498 , 99
養育 , 87 , 10
為Y作哀辭 , 167 , 511
恩主是Y , 5 , 132
為Y的墓誌銘作額篆 , 532 , 

In [3]:
from collections import defaultdict
def statistic_social_status():
    '''
    统计社会地位
    PersonEntryInfo"->"Entry"->"RuShiDoor" : "正常科舉",
    PersonPostings" -> PersonSocialStatus"-> "SocialStatus"
    '''
    entry_dict = defaultdict(int)
    social_status_dict = defaultdict(int)
    error = 0
    for fname in os.listdir(DATA_DIR):
        fpath = os.path.join(DATA_DIR, fname)
        with open(fpath) as f:
            try:
                json_data = json.load(f)
                # print(json_data)
                if 'Person' not in json_data['Package']['PersonAuthority']['PersonInfo']:
                    continue
                person_data = json_data['Package']['PersonAuthority']['PersonInfo']['Person']
                if 'PersonEntryInfo' in person_data and 'Entry' in person_data['PersonEntryInfo']:
                    tmp = person_data['PersonEntryInfo']['Entry']
                    if isinstance(tmp, list):
                        for tmp2 in tmp:
                            if 'RuShiDoor' in tmp2:
                                tmp3 = tmp2['RuShiDoor']
                                entry_dict[tmp3] += 1
                    else:
                        tmp3 = tmp['RuShiDoor']
                        entry_dict[tmp3] += 1
                if 'PersonSocialStatus' in person_data and 'SocialStatus' in person_data['PersonSocialStatus']:
                        tmp = person_data['PersonSocialStatus']['SocialStatus']
                        if isinstance(tmp, list):
                            for tmp2 in tmp:
                                tmp3 = tmp2['StatusName']
                                social_status_dict[tmp3] += 1
                        else:
                            tmp3 = tmp['StatusName']
                            social_status_dict[tmp3] += 1
            except json.decoder.JSONDecodeError as e:
                    error += 1
    print(error)
    print(entry_dict)
    print(social_status_dict)
statistic_social_status()

defaultdict(<class 'int'>, {'': 4343, '監生門': 3407, '宗教門': 42, '血親門': 1157, '姻親門': 47, '徵召門': 1137, '其他舉': 74, '求官不得門': 177, '正常科舉': 75716, '宮廷門': 278, '貢生門': 1424, '特旨門': 1910, '恩蔭門': 1712, '未知入仕途徑': 306, '進納門': 31, '非經常科舉': 311, '軍功補授門': 275, '歸降門': 43, 'Temp': 4, '其他入仕途徑': 78, '技進門': 29, '薦舉門': 681, '七色補官門': 32, '制舉': 441, '學生門': 2671}) defaultdict(<class 'int'>, {'': 5, '儒學': 305, '經學家—公羊學': 8, '[節度使]': 15, '書院山長': 295, '未仕而卒': 275, '工於古文': 104, '妾': 39, '義軍首領': 118, '工匠': 9, '父老;耆老': 24, '酷吏': 15, '[學生： 八行]': 8, '理學家': 713, '奴婢': 7, '落第士人': 275, '[]': 2, '醫學教授': 6, '[母]': 595, '教師：軍事教官': 40, '並稱/齊名': 16, '[正黃旗]': 5, '業進士': 140, '經學家—禮經': 129, '道士': 272, '好學': 354, '在室女': 4, '書法家': 872, '未婚而卒': 76, '建書院/義塾/家塾  ': 52, '[有官銜]': 239, '[鑲藍旗]': 1, '[刺史幕僚（知州幕僚）]': 6, '曲作家': 25, '天文學家（星象家）': 110, '[未婚妻]': 4, '[武學生]': 2, '作家': 2, '削籍官員': 176, '薩滿教巫師': 1, '目錄學家': 3, '主家事': 50, '[武官]': 2473, '酋長': 34, '精於戲劇': 9, '[皇帝]': 178, '[舉人]': 17, '腐儒': 1, '隱居授徒': 105, '刻工': 2, '門客': 38, '道姑(女冠)': 14, '